In [20]:
import pprint
import io
from io import StringIO
import json
import pandas as pd
import logging
from datetime import datetime
import csv
from collections import defaultdict

## Parsing GFF

I have tried many methods:

* gffutils- waited an hour for the db to build and gave up.
* BCBio it parsed the file OK, but I couldn't figure out how to get the features I wanted. 

Will reflex to using NCBI gene2refseq file which I htink will get me what I need, though you can only get 1 file for all of the refseq taxonomy. Pulling out human info and we'll see if this works. q

In [29]:
hs_annot='../data/references/human_gene2refseq'
id_list='../data/processed/gid_list.txt'

In [30]:
gid_df=pd.read_csv(id_list, header=None)
display(gid_df.head())

,0
0,1
1,2
2,100129239
3,131076
4,9


In [31]:
gid=gid_df.iloc[:, 0].tolist()
print(gid)

[1, 2, 100129239, 131076, 9, 10, 12, 13, 14, 15, 16, 729447, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 163882, 43, 131118, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 70, 71, 72, 131149, 81, 100507003, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 163933, 97, 98, 100, 101, 102, 103, 104, 105, 131177, 107, 108, 109, 338692, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 123, 124, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 196740, 196743, 140, 141, 142, 143, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 644809, 100129271, 173, 174, 175, 176, 177, 178, 220134, 181, 182, 183, 196792, 185, 186, 187, 189, 190, 191, 644815, 196, 197, 199, 202, 203, 204, 205, 164045, 207, 208, 100507027, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 149840, 229, 230, 231, 238, 239, 240, 241, 242, 246, 247, 248, 249, 250, 251, 164091, 257, 258, 

In [37]:
gid2transcript=defaultdict(list)

with open(hs_annot, 'r') as infile:
    csv_reader = csv.reader(infile, delimiter='\t')
    for row in csv_reader:
        if row[0].startswith("#"):
            continue
        if row[12].startswith('Alternate'):
            continue
        gid=row[1]
        if row[3] != '-':
            #print(row[3])
            if gid not in gid2transcript:
                gid2transcript[gid] = []
            gid2transcript[gid].append(row[3])

print(len(gid2transcript))

43312


In [38]:
gid2txnum={}
for gid in gid2transcript:
    gid2txnum[gid]=len(gid2transcript[gid])

print(gid2txnum)

{'1': 1, '2': 6, '3': 1, '9': 16, '10': 3, '12': 5, '13': 2, '14': 3, '15': 4, '16': 3, '18': 25, '19': 2, '20': 5, '21': 2, '22': 7, '23': 15, '24': 4, '25': 4, '26': 6, '27': 14, '28': 4, '29': 23, '30': 7, '31': 47, '32': 19, '33': 4, '34': 7, '35': 3, '36': 3, '37': 11, '38': 16, '39': 3, '40': 3, '41': 13, '43': 13, '47': 6, '48': 4, '49': 2, '50': 2, '51': 7, '52': 6, '53': 9, '54': 10, '55': 6, '56': 16, '58': 2, '59': 13, '60': 2, '70': 7, '71': 7, '72': 3, '81': 11, '86': 3, '87': 38, '88': 4, '89': 4, '90': 8, '91': 24, '92': 4, '93': 6, '94': 13, '95': 6, '97': 7, '98': 6, '100': 5, '101': 8, '102': 3, '103': 17, '104': 30, '105': 1, '107': 5, '108': 6, '109': 31, '111': 19, '112': 13, '113': 33, '114': 5, '115': 4, '116': 4, '117': 11, '118': 25, '119': 12, '120': 32, '123': 3, '124': 1, '125': 3, '126': 3, '127': 3, '128': 1, '130': 3, '131': 2, '132': 16, '133': 1, '134': 6, '135': 8, '136': 7, '140': 4, '141': 6, '142': 1, '143': 1, '146': 1, '147': 8, '148': 14, '150': 

In [39]:
#write to disc
with open('../data/processed/gene_isoform.json', 'w') as json_file:
    json.dump(gid2txnum, json_file)